### Import libraries and define functions

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import itertools
from pathlib import Path

np.set_printoptions(precision=3, linewidth=120, suppress=True)

In [ ]:
def calc_lambda(A, func=None, addremove=1, p=False):
    """Calculate the current lambda_2 and K_lambda_2 and return them."""
    D = np.diag(A.sum(1))
    L = D - A
    lambdas, vectors = np.linalg.eigh(L)
    sort = lambdas.argsort()
    lambdas = lambdas[sort]
    vectors = vectors[:,sort]

    l2 = lambdas[1]
    f = vectors[:,1]
    l2_multiplicity = np.count_nonzero(np.isclose(lambdas, l2))

    K_l2 = 0
    s_val = 0
    pairs = None
    if func is not None:
        search = np.empty_like(A)
        search[:] = np.nan
        for i, j in itertools.combinations(range(len(f)), 2):
            if abs(f[i] - f[j]) > 10e-5 and ((addremove == 1 and A[i][j] == 0) or (addremove == -1 and A[i][j] == 1)):
                search[i][j] = addremove * (f[i] - f[j]) ** 2

            if A[i][j] == 1:
                K_l2 = max(K_l2, (f[i] - f[j]) ** 2)

        s_val = func(search)
        if l2_multiplicity == 1 or addremove == -1:
            pairs = {tuple(p) for p in np.argwhere(np.isclose(search, s_val))}
        else:
            pairs = f'Unknown, at least {l2_multiplicity} links'



    if p:
        print(f"({l2:.5f}, {K_l2:.5f})")
        print(f"L  = {np.array2string(L, prefix='L  = ')}")
        print(f"eval={np.array2string(lambdas)}")
        print(f"evec={np.array2string(vectors, prefix='evec=')}")
        print('-----------------------------------------------\n')

    return round(l2, 5), round(K_l2 * 0.2, 5), pairs, f

In [ ]:
def plot_graphs(graphs, figsize=14, dotsize=20):
    """Utility to plot a lot of graphs from an array of graphs.
    Each graphs is a list of edges; each edge is a tuple."""
    fig = plt.figure(figsize=(figsize,figsize))
    fig.patch.set_facecolor('white') # To make copying possible (no transparent background)
    k = int(np.sqrt(len(graphs)))
    for i, g in enumerate(graphs):
        plt.subplot(k+1,k+1,i+1)
        G = nx.from_numpy_array(graphs[g])
        nx.draw_kamada_kawai(G, node_size=dotsize)
        plt.title(f"Graph {g} - {len(G.edges)}\n{calc_lambda(graphs[g], np.nanmin)[:2]}")
        print('.', end='')

#plot_graphs([[(0,1),(1,2),(1,3)]])

### Load graphs and plot them

In [ ]:
from collections import OrderedDict

# Make the graphs and plot them.
NV = 6  # Number of nodes in the graph.
data_folder = Path.cwd() / '../Data'


with open(data_folder / f'UniqueGraphs_{NV}.npz', 'rb') as stream:
    loaded = np.load(stream)
    gs = OrderedDict(loaded)

print(f'Drawing {len(gs)} graphs...')
plot_graphs(gs, figsize=30, dotsize=20)


### Possible values of lambda_2 depending on the number of edges

In [ ]:
from collections import defaultdict

l2_by_edges = defaultdict(list)

for i, A in enumerate(gs.values()):
    # print(f"Graph {i}", end=' ')
    l2 = calc_lambda(A, None)[0]
    l2_by_edges[np.count_nonzero(A == 1) / 2].append(l2)
    


In [ ]:
labels = sorted(l2_by_edges.keys())
data_len = max(len(v) for v in l2_by_edges.values())
data = [[0 for _ in range(len(labels))] for _ in range(data_len)]

for i, lab in enumerate(labels):
    for j, v in enumerate(sorted(l2_by_edges[lab])):
        data[j][i] = v

        
fig, (ax1, ax2, ax3) = plt.subplots(3, 1)
fig.set_size_inches(16, 18) 
rects = []

# 1st axes
x = np.arange(len(labels))  # the label locations
width = 0.9  # the width of the bars
for i in range(data_len):
    rec = ax1.bar(x + i * width / data_len, data[i], width / data_len)
    ax1.bar_label(rec, labels=[f'{v:.3f}' if v > 0 else '' for v in data[i]],
                padding=3, fmt='%.3f', rotation='vertical')
ax1.set_title('Possible $\lambda_2$ by number of edges')
ax1.set_ylabel('$\lambda_2$')
ax1.set_xlabel('Num edges')
ax1.set_xticks(x, labels)
ax1.grid(axis='y')

# 2nd axes - line plot
possible_l2 = sorted(set(l2 for temp in data for l2 in temp))
max_l2 = possible_l2[-1]
x = np.linspace(0, max_l2, 500)
y = []
i = 1
for xi in x:
    while xi > possible_l2[i]:
        i += 1
    y.append(min(xi - possible_l2[i-1], possible_l2[i] - xi))
ax2.plot(x, y)
ax2.set_title('Needed $K_{\lambda_2}$ for specified $\lambda_2$')
ax2.set_ylabel('Minimum $K_{\lambda_2}$')
ax2.set_xlabel('Reference $\lambda_2$')
ax2.grid()

# 2nd axes - error bars
#possible_l2 = sorted(set(l2 for temp in data for l2 in temp))
#print(possible_l2)
#max_l2 = possible_l2[-1]
#x = np.linspace(0, max_l2, 500)
#y = x

#i = 1
#yerr = []
#for xi in x:
#    if xi > possible_l2[i]:
#        i += 1
#    yerr.append(min(xi - possible_l2[i-1], possible_l2[i] - xi))
#ax2.errorbar(x, y, yerr=yerr)

# 3rd axes - scatter plot
y = np.ones((1, len(possible_l2)))
x = possible_l2
ax3.scatter(x, y, s=20)
ax3.set_title('Possible values of $\lambda_2$')
ax3.set_xlabel('$\lambda_2$')

### Analyze lambda_2 changes when changing link values

In [ ]:
import plotly.graph_objects as pgo
import plotly.subplots as psub
from Utils.plotting_funcs import plotly_nx


def make_analysis(k, A_start, p=False):
    x = np.linspace(0, 1, 50)
    N = A_start.shape[0]

    # Initial lambda value and max change estimate.
    init_lambda, addmax_est, addmax_links, f = calc_lambda(A_start, np.nanmax, addremove=1, p=False)
    _, addmin_est, addmin_links, _ = calc_lambda(A_start, np.nanmin, addremove=1, p=False)
    _, remmax_est, remmax_links, _ = calc_lambda(A_start, np.nanmax, addremove=-1, p=False)
    _, remmin_est, remmin_links, _ = calc_lambda(A_start, np.nanmin, addremove=-1, p=False)

    sens_data = {}
    trend = {}
    for i in range(0, N):
        for j in range(i + 1, N):
            sens_data[(i,j)] = []
            A = A_start.copy()

            # Calculate continuous changes in l2 when changing links.
            for t in x:
                A[i][j] = t
                A[j][i] = t
                l2 = calc_lambda(A, None)[0]
                sens_data[(i, j)].append(l2)
                trend[(i, j)] = 'dash' if A_start[i][j] else 'solid'

            # Something else

    # Make plots.
    fig = psub.make_subplots(rows=1, cols=2)
    nodes, edges = plotly_nx(nx.from_numpy_array(A_start))
    fig.add_trace(nodes, row=1, col=1)
    fig.add_trace(edges, row=1, col=1)
    for edge in sens_data:
        fig.add_trace(
            pgo.Scatter(x=x, y=sens_data[edge], name=str(edge),
                        mode='lines', line=dict(dash=trend[edge])),
            row=1, col=2
        )
    fig.update_layout(title=dict(text=f'Graph {k} (l2 = {init_lambda}, f = {f})'),
                      width=750, height=600,
                      margin=dict(l=50, r=50, t=80, b=200),
                      )

    # Print results.
    def gen_annot(est, links,):
        return (f'Estimated delta = {est}<br>'
                f'Links maximizing delta = {links}<br>')
    annot_kwargs = dict(align='left', showarrow=False, xref='paper', yref='paper')

    fig.add_annotation(text="<b>Adding max link</b><br>" + gen_annot(addmax_est, addmax_links), x=0.0, y=-0.3, **annot_kwargs)
    fig.add_annotation(text="<b>Adding min link</b><br>" + gen_annot(addmin_est, addmin_links), x=0.0, y=-0.5, **annot_kwargs)
    fig.add_annotation(text="<b>Removing max link</b><br>" + gen_annot(remmax_est, remmax_links), x=1, y=-0.3, **annot_kwargs)
    fig.add_annotation(text="<b>Removing min link</b><br>" + gen_annot(remmin_est, remmin_links), x=1, y=-0.5, **annot_kwargs)
    fig.show()


for k, A_start in enumerate(gs.values()):
    make_analysis(k, A_start)

### Check the consistency of $$\Delta \lambda_{2, max} = \max(f_i - f_j)^2$$

In [ ]:
from collections import Counter
count = Counter()


for k, A_start in enumerate(gs):
    if k in []:
        debug = True
    else:
        debug = False
    init_lambda, delta_est, init_links, _ = calc_lambda(A_start, np.nanmax, addremove=1, p=debug)

    n = A_start.shape[0]
    max_lambda = init_lambda
    max_links = set()
    for i in range(0, NV):
        for j in range(i + 1, NV):
            if A_start[i][j] == 0:
                A = A_start.copy()
                A[i][j] = 1
                A[j][i] = 1

                new_lambda = calc_lambda(A, np.nanmax)[0]
                if (new_lambda - max_lambda) > 10e-5:
                    max_lambda = new_lambda
                    max_links = {(i, j)}
                elif abs(new_lambda - max_lambda) < 10e-5 and abs(new_lambda - init_lambda) > 10e-5:
                    max_links.add((i, j))

    print(f'Graph {k}...', end=' ')

    if max_links == init_links or not max_links or init_links.issubset(max_links):
        out = 'OK'
    elif max_links.issubset(init_links):
        out = 'Partial'
    else:
        out = 'FAIL'
        make_analysis(k, A_start)
    count[out] += 1

    print(f'{out} (l2o={init_lambda}, l2f={max_lambda}, est={init_links}, act={max_links})', flush=True)

total = sum(count.values())
print('=========================')
print(f"OK     : {count['OK']}/{total} [{count['OK'] / total * 100:.2f} %]")
print(f"Partial: {count['Partial']}/{total} [{count['Partial'] / total * 100:.2f} %]")
print(f"FAIL   : {count['FAIL']}/{total} [{count['FAIL'] / total * 100:.2f} %]")

In [ ]:
import time

def select_link(A, addremove):
    L = np.diag(A.sum(1)) - A
    lambdas, vectors = np.linalg.eigh(L)
    sort = lambdas.argsort()
    lambdas = lambdas[sort]
    vectors = vectors[:, sort]

    l2 = lambdas[1]
    f = vectors[:, 1]

    search = []
    f = np.around(f, 4)
    for i, j in itertools.combinations(range(len(f)), 2):
        if abs(f[i] - f[j]) > 10e-5 and ((addremove == "ADD" and A[i][j] == 0) or (addremove == "REM" and A[i][j] == 1)):
            search.append((round((f[i] - f[j]) ** 2, 4), (i, j)))

    if not search:
        return None

    if addremove == "ADD":
        f_val, link = max(search)
    else:
        f_val, link = min(search)

    return l2, f_val, link


starting_graph = 'G6,102'
ref_lambda = [1, 3, 2]

data_folder = Path.cwd() / '../Data'
with open(data_folder / f'UniqueGraphs_{starting_graph[1]}.npz', 'rb') as stream:
    loaded = np.load(stream)
    graphs = {x: loaded[x] for x in loaded}

k = 0
A = graphs[starting_graph]
history = [A.copy()]
addremove = "ADD"
l2 = 0

for ref in ref_lambda:
    user_input = 'Y'

    print(f'ref={ref}')
    while user_input != 'n':
        l2, _, _, = select_link(A, addremove)

        if l2 < ref - 0.2:
            addremove = "ADD"
        elif l2 > ref + 0.2:
            addremove = "REM"
        else:
            break

        _, _, pair, = select_link(A, addremove)
        val = 1 if addremove == "ADD" else 0
        A[pair[0]][pair[1]] = val
        A[pair[1]][pair[0]] = val
        history.append(A.copy())

        print(f"k={k} | l2={l2:6.4f} {addremove} {pair}' > Continue? (Y/n)", flush=True)
        time.sleep(0.5)
        user_input = input()

        k += 1


for k, A in enumerate(history):
   make_analysis(k, A)